# **¿Enemigo #1? Global Interpreter Lock en Python**

In [2]:
import threading

def tarea():
    for _ in range(3):
        print("Hola desde un hilo")

# Crear dos hilos
hilo1 = threading.Thread(target=tarea)
hilo2 = threading.Thread(target=tarea)

hilo1.start()
hilo2.start()

hilo1.join()
hilo2.join()


Hola desde un hilo
Hola desde un hilo
Hola desde un hilo
Hola desde un hilo
Hola desde un hilo
Hola desde un hilo


In [4]:
from multiprocessing import Process

def tarea():
    for _ in range(3):
        print("Hola desde un proceso")

# Crear dos procesos
p1 = Process(target=tarea())
p2 = Process(target=tarea())

p1.start()
p2.start()

p1.join()
p2.join()


Hola desde un proceso
Hola desde un proceso
Hola desde un proceso
Hola desde un proceso
Hola desde un proceso
Hola desde un proceso


In [19]:
# threading_timeline.py
import os, sys, time, datetime as dt, threading

# ---- obtener el core actual (Windows/Linux) ----
def current_core():
    if hasattr(os, "sched_getcpu"):
        try:
            return os.sched_getcpu()
        except Exception:
            pass
    if sys.platform.startswith("win"):
        try:
            import ctypes
            GetCurrentProcessorNumber = ctypes.windll.kernel32.GetCurrentProcessorNumber
            GetCurrentProcessorNumber.restype = ctypes.c_uint
            return int(GetCurrentProcessorNumber())
        except Exception:
            pass
    return None

print_lock = threading.Lock()
events = []  # (ts_abs, tag, iter, phase, core, tid)

t0 = time.perf_counter()  # referencia común para todos

def ms(ts):  # helper: segundos -> ms relativos a t0
    return (ts - t0) * 1000.0

def trabajo(tag, iters=6, carga=450_000):
    tid = threading.get_native_id()
    pid = os.getpid()
    for k in range(iters):
        ts_in = time.perf_counter()
        core_in = current_core()
        with print_lock:
            print(f"[{ms(ts_in):9.3f} ms] START {tag:<8} | iter={k} | pid={pid} | tid={tid} | core={core_in}", flush=True)
        events.append((ts_in, tag, k, "START", core_in, tid))

        # --- carga CPU para que el scheduler alterne hilos ---
        x = 0
        for _ in range(carga):
            x += 1
        time.sleep(0.02)

        ts_out = time.perf_counter()
        core_out = current_core()
        dur = (ts_out - ts_in) * 1000.0
        with print_lock:
            print(f"[{ms(ts_out):9.3f} ms] END   {tag:<8} | iter={k} | pid={pid} | tid={tid} | core={core_out} | dur={dur:7.3f} ms", flush=True)
        events.append((ts_out, tag, k, "END", core_out, tid))

if __name__ == "__main__":
    h1 = threading.Thread(target=trabajo, name="Hilo-A", args=("thread-A",))
    h2 = threading.Thread(target=trabajo, name="Hilo-B", args=("thread-B",))
    h1.start(); h2.start()
    h1.join(); h2.join()

    # ---- Resumen: línea de tiempo global ordenada ----
    events.sort(key=lambda e: e[0])
    print("\n=== Timeline (orden por tiempo, ms desde t0) ===")
    for ts_abs, tag, k, phase, core, tid in events:
        print(f"{ms(ts_abs):9.3f} ms | {phase:5} | {tag:<8} | iter={k} | tid={tid} | core={core}")


[    0.959 ms] START thread-A | iter=0 | pid=7804 | tid=24408 | core=4
[   18.426 ms] START thread-B | iter=0 | pid=7804 | tid=28716 | core=8
[   48.425 ms] END   thread-A | iter=0 | pid=7804 | tid=24408 | core=2 | dur= 47.466 ms
[   49.564 ms] START thread-A | iter=1 | pid=7804 | tid=24408 | core=0
[   61.485 ms] END   thread-B | iter=0 | pid=7804 | tid=28716 | core=4 | dur= 43.059 ms
[   62.189 ms] START thread-B | iter=1 | pid=7804 | tid=28716 | core=8
[   79.686 ms] END   thread-A | iter=1 | pid=7804 | tid=24408 | core=6 | dur= 30.122 ms
[   80.201 ms] START thread-A | iter=2 | pid=7804 | tid=24408 | core=0
[   92.083 ms] END   thread-B | iter=1 | pid=7804 | tid=28716 | core=4 | dur= 29.894 ms
[   92.653 ms] START thread-B | iter=2 | pid=7804 | tid=28716 | core=0
[  110.465 ms] END   thread-A | iter=2 | pid=7804 | tid=24408 | core=6 | dur= 30.264 ms
[  111.098 ms] START thread-A | iter=3 | pid=7804 | tid=24408 | core=4
[  123.107 ms] END   thread-B | iter=2 | pid=7804 | tid=28716 |